In [ ]:
import streamlit as st
import pandas as pd
import mysql.connector as msql
from mysql.connector import Error
import plotly.express as px
import geopandas as gpd
from streamlit_option_menu import option_menu
import requests
from streamlit_lottie import st_lottie


try:
    conn = msql.connect(host='localhost',
                        database='Phonepe_Pulse', 
                        user='root',
                        password='1234')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM agg_tran_state")
        record1 = cursor.fetchall()
        agg_tran_state = pd.DataFrame(record1,columns=[i[0] for i in cursor.description])

        cursor.execute("SELECT * FROM agg_tran_user")
        record2 = cursor.fetchall()
        agg_tran_user = pd.DataFrame(record2,columns=[i[0] for i in cursor.description])

        cursor.execute("SELECT * FROM map_tran_state")
        record3 = cursor.fetchall()
        map_tran_state = pd.DataFrame(record3,columns=[i[0] for i in cursor.description])

        cursor.execute("SELECT * FROM map_tran_user")
        record4 = cursor.fetchall()
        map_tran_user = pd.DataFrame(record4,columns=[i[0] for i in cursor.description])

        conn.commit()
        cursor.close()
        conn.close()



except Error as e:
    print("error",e)


with st.sidebar:
    menu = option_menu(menu_title ="main menu", 
                       options=['HOME','APP REGISTERED','USER MOBILE BRAND','BRAND PERCENTAGE'],
                       icons=['home','app','phone','pie-chart'],
                       default_index=0)
    

if menu == 'HOME':
    st.markdown("<h1 style='text-align:center; color:purple;'"
                ">Phonepe Pulse Data Visualization and Exploration</h1>",
                unsafe_allow_html=True)
    
    st.image("phonepe.jpg")
    st.write("PhonePe is a leading digital payments and financial technology platform that has revolutionized the way people in India manage their financial transactions. Established in 2015, PhonePe has rapidly grown to become one of the most popular and trusted mobile payment solutions in the country."

             "At its core, PhonePe serves as a comprehensive financial ecosystem, offering a range of services that extend beyond traditional payment processing. The platform enables users to make seamless and secure transactions, including money transfers, bill payments, recharges, and online purchases. Its user-friendly interface and widespread acceptance make it a go-to choice for millions of users seeking convenience and efficiency in their financial transactions."

             "One of PhonePe's key strengths is its UPI (Unified Payments Interface) integration, allowing users to link their bank accounts directly to the app. This feature facilitates quick and direct transfers between individuals and merchants, eliminating the need for traditional banking methods. The UPI infrastructure also plays a pivotal role in fostering financial inclusion by providing easy access to digital payments for users across various demographics."

             "PhonePe goes beyond basic payment services, offering innovative solutions such as PhonePe Switch, which integrates a variety of third-party apps and services within the platform. Users can seamlessly access food delivery, travel bookings, entertainment, and more, all through the PhonePe app. This consolidation of services enhances user engagement and positions PhonePe as a multifaceted solution for daily needs."

             "The platform's commitment to security and privacy is evident in its robust encryption measures, ensuring the protection of user data and financial information. PhonePe has gained the trust of users by adhering to stringent security standards and continuously investing in technology to stay ahead of potential threats."

             "As a pioneer in the Indian digital payments landscape, PhonePe has not only simplified financial transactions but has also played a crucial role in driving the nation's transition towards a cashless economy. With constant innovation, user-centric features, and a commitment to excellence, PhonePe continues to shape the future of digital finance in India.")

    
yearTuple = ('2018', '2019', '2020', '2021', '2022')
quaterTuple = ('Q1', 'Q2', 'Q3', 'Q4')
stateTuple= ('andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
                 'assam', 'bihar', 'chandigarh', 'chhattisgarh',
                 'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
                 'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
                 'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
                 'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
                 'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
                 'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
                 'uttarakhand', 'west-bengal')


if menu == "APP REGISTERED":
    st.markdown("<h1 style='text-align:center;color:red;'"
                ">App Registered User By District</h1>",unsafe_allow_html=True)
    
    Year = st.selectbox('SELECT THE YEAR:',yearTuple)
    st.write(' ')
    State = st.selectbox('SELECT THE STATE:',stateTuple,index=30)

    map_tran_user_filter = map_tran_user[(map_tran_user['state']==State)&(map_tran_user['year']==int(Year))]
    dist_reg = map_tran_user_filter.groupby(['district']).sum(numeric_only = True)[['registereduser','appopens']]
    dist_reg = dist_reg.reset_index()

    fig1 = px.bar(dist_reg,
                  x="district",
                  y="registereduser",
                  color='district',
                  title=f"DISTRICT WISE REGISTERED USER IN {State}:")
    fig1.update_traces(width=1)
    st.plotly_chart(fig1)                                                                                                                            


if menu == "USER MOBILE BRAND":

    st.markdown("<h1 style='text-align:center; color:red;'"
                ">Mobile Brand Analysis By State</h1>",unsafe_allow_html=True)


    StateBar = st.selectbox('Please select State', stateTuple, index=30)
    yearBar = st.radio('Please select the Year:', yearTuple, horizontal=True)
    QuaterBar = st.radio('Please select the Quarter:', quaterTuple, horizontal=True)

    user_filter = agg_tran_user[(agg_tran_user['state'] == StateBar)&(agg_tran_user['year'] == int(yearBar))&(agg_tran_user['quater'] == QuaterBar)]

    userbrand = px.bar(user_filter,
                       x='brand',
                       y='brand_count',
                       color='brand',
                       title='USER MOBILE BRAND ANALYSIS',
                       color_continuous_scale='magma')
    
    st.plotly_chart(userbrand)

if menu == "BRAND PERCENTAGE":
    st.markdown("<h1 style='text-align:center; color:red;'"
                ">Mobile Brand Percentage Analysis</h1>",
                unsafe_allow_html=True)
   
    StatePie = st.selectbox('Please Choose State', stateTuple, index=30)
    yearPie = st.radio('Please Choose the Year:', yearTuple, horizontal=True)
    QuaterPie = st.radio('Please choose the Quarter:', quaterTuple, horizontal=True)

    User_filterPie = agg_tran_user[(agg_tran_user['state'] == StatePie)
                                        & (agg_tran_user['year'] == int(yearPie))
                                        & (agg_tran_user['quater'] == QuaterPie)]
    

    BrandPercent = px.pie(User_filterPie,
                       names='brand',
                       values='brand_percentage',
                       color='brand',
                       template='plotly_dark',
                       title='User Mobile Brand in percentage ',
                       width=800,
                       height=600)

    BrandPercent.update_traces(textposition='inside',
                               textinfo='percent+label',
                               textfont_size=15,
                               insidetextorientation='radial',
                               pull=[0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                               marker=dict(line=dict(color='#000000', width=2)))

    BrandPercent.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')

    st.plotly_chart(BrandPercent)
